## Lesson 5: Human in the Loop

https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/6/human-in-the-loop

In [33]:
import operator
from typing import Annotated, TypedDict
from uuid import uuid4

from dotenv import find_dotenv, load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph

In [34]:
load_dotenv(find_dotenv())

MODEL = "glm-4-air"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

In [35]:
memory = SqliteSaver.from_conn_string(":memory:")

In [36]:
"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""


def reduce_messages(
    left: list[AnyMessage], right: list[AnyMessage]
) -> list[AnyMessage]:
    for message in right:
        if not message.id:
            message.id = str(uuid4())

    merged = left.copy()

    for message in right:
        for i, existing in enumerate(merged):
            if message.id == existing.id:
                merged[i] = message
                break
        else:
            merged.append(message)

    return merged


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [37]:
tool = TavilySearchResults(max_results=2)

### Manual human approval

In [38]:
class Agent:
    def __init__(self, model, tools: list, system="", checkpointer=None) -> None:
        self.system = system

        graph = StateGraph(state_schema=AgentState)
        graph.add_node(node="llm", action=self.call_openai)
        graph.add_node(node="action", action=self.take_action)
        graph.add_conditional_edges(
            source="llm", path=self.exists_action, path_map={True: "action", False: END}
        )
        graph.add_edge(start_key="action", end_key="llm")
        graph.set_entry_point(key="llm")
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )

        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState) -> dict:
        messages = state["messages"]

        if self.system:
            messages = [SystemMessage(content=self.system)] + messages

        message = self.model.invoke(messages)

        return {"messages": [message]}

    def exists_action(self, state: AgentState) -> bool:
        print(state)

        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState) -> dict:
        tool_calls = state["messages"][-1].tool_calls
        results = []

        for t in tool_calls:
            print(f"Calling tool: {t}")

            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )

        print("Back to the Model!")

        return {"messages": results}

In [39]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [40]:
model = ChatOpenAI(model=MODEL, base_url=BASE_URL)

abot = Agent(model=model, tools=[tool], system=prompt, checkpointer=memory)

In [11]:
messages = [HumanMessage(content="What is the weather like in SF now?")]

thread = {"configurable": {"thread_id": 1}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='What is the weather like in SF now?', id='acd7beb9-309b-44d6-b714-260084989e91'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256029470882841', 'function': {'arguments': '{"query": "San Francisco current weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 217, 'total_tokens': 234}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-98814db1-f259-4e90-9f95-acbfcbe6785f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'call_8726256029470882841'}], usage_metadata={'input_tokens': 217, 'output_tokens': 17, 'total_tokens': 234})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256029470882841', 'function': {'arguments': '{"query": "San Francisco current w

In [12]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather like in SF now?', id='acd7beb9-309b-44d6-b714-260084989e91'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256029470882841', 'function': {'arguments': '{"query": "San Francisco current weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 217, 'total_tokens': 234}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-98814db1-f259-4e90-9f95-acbfcbe6785f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'call_8726256029470882841'}], usage_metadata={'input_tokens': 217, 'output_tokens': 17, 'total_tokens': 234})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2490a-7cf3-67c8-8001-d31d5aadf34d'}}, metadata={'source': 'loop', 

In [13]:
abot.graph.get_state(thread).next

('action',)

### continue after interrupt

In [14]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'call_8726256029470882841'}
Back to the Model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1717739013, \'localtime\': \'2024-06-06 22:43\'}, \'current\': {\'last_updated_epoch\': 1717738200, \'last_updated\': \'2024-06-06 22:30\', \'temp_c\': 13.3, \'temp_f\': 55.9, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 8.1, \'wind_kph\': 13.0, \'wind_degree\': 310, \'wind_dir\': \'NW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.84, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 93, \'cloud\': 0, \'feelslike_c\': 12.1, \'feel

In [15]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather like in SF now?', id='acd7beb9-309b-44d6-b714-260084989e91'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256029470882841', 'function': {'arguments': '{"query": "San Francisco current weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 217, 'total_tokens': 234}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-98814db1-f259-4e90-9f95-acbfcbe6785f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'call_8726256029470882841'}], usage_metadata={'input_tokens': 217, 'output_tokens': 17, 'total_tokens': 234}), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'Californ

In [16]:
abot.graph.get_state(thread).next

()

In [18]:
messages = [HumanMessage(content="What is the weather like in LA now?")]

thread = {"configurable": {"thread_id": 2}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='What is the weather like in LA now?', id='b89dbc41-2c8b-44d4-97c8-01fe94c1e734'), HumanMessage(content='What is the weather like in LA now?', id='ad63ff78-1b77-4953-a012-f5ef7a0da637'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726260083920209225', 'function': {'arguments': '{"query": "LA current weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 228, 'total_tokens': 244}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d17f39c4-ec69-4975-b8b5-904038028106-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA current weather'}, 'id': 'call_8726260083920209225'}], usage_metadata={'input_tokens': 228, 'output_tokens': 16, 'total_tokens': 244})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8

In [19]:
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread), "\n")

    _input = input("Proceed?")

    if _input != "y":
        print("Bye!")
        break

    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)


 StateSnapshot(values={'messages': [HumanMessage(content='What is the weather like in LA now?', id='b89dbc41-2c8b-44d4-97c8-01fe94c1e734'), HumanMessage(content='What is the weather like in LA now?', id='ad63ff78-1b77-4953-a012-f5ef7a0da637'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726260083920209225', 'function': {'arguments': '{"query": "LA current weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 228, 'total_tokens': 244}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d17f39c4-ec69-4975-b8b5-904038028106-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA current weather'}, 'id': 'call_8726260083920209225'}], usage_metadata={'input_tokens': 228, 'output_tokens': 16, 'total_tokens': 244})]}, next=('action',), config={'configurable': {'thread_id': '2',

### Modify State

Run until the interrupt and then modify the state.

In [41]:
messages = [HumanMessage(content="What is the weather like in LA now?")]

thread = {"configurable": {"thread_id": 3}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='What is the weather like in LA now?', id='87141516-c120-4c37-9d1c-aba077de68cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 217, 'total_tokens': 235}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7433f4af-3e19-44a6-928c-337cc9fff2f2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_8726256235629355699'}], usage_metadata={'input_tokens': 217, 'output_tokens': 18, 'total_tokens': 235})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Lo

In [42]:
current_values = abot.graph.get_state(thread)

In [43]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Los Angeles'},
  'id': 'call_8726256235629355699'}]

In [44]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]

current_values.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in Louisiana"},
        "id": _id,
    }
]

In [45]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='What is the weather like in LA now?', id='87141516-c120-4c37-9d1c-aba077de68cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 217, 'total_tokens': 235}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7433f4af-3e19-44a6-928c-337cc9fff2f2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_8726256235629355699'}], usage_metadata={'input_tokens': 217, 'output_tokens': 18, 'total_tokens': 235})]}


{'configurable': {'thread_id': '3',
  'thread_ts': '1ef24b26-a960-65e7-8002-e7d0018028b3'}}

In [46]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather like in LA now?', id='87141516-c120-4c37-9d1c-aba077de68cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 217, 'total_tokens': 235}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7433f4af-3e19-44a6-928c-337cc9fff2f2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_8726256235629355699'}], usage_metadata={'input_tokens': 217, 'output_tokens': 18, 'total_tokens': 235})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef24b26-a960-65e7-8002-e7d0018028b3'}}, metadata={'source': 'update'

In [47]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_8726256235629355699'}
Back to the Model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.44, \'lon\': -91.06, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1717753390, \'localtime\': \'2024-06-07 4:43\'}, \'current\': {\'last_updated_epoch\': 1717752600, \'last_updated\': \'2024-06-07 04:30\', \'temp_c\': 17.2, \'temp_f\': 63.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, \'wind_degree\': 280, \'wind_dir\': \'W\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.9, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 54, \'cloud\': 0, \'feelslike_c\': 17.2, \'feelslike_f\': 6

### Time Travel

In [48]:
states = []

for state in abot.graph.get_state_history(thread):
    print(state)
    print("------")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather like in LA now?', id='87141516-c120-4c37-9d1c-aba077de68cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 217, 'total_tokens': 235}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7433f4af-3e19-44a6-928c-337cc9fff2f2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_8726256235629355699'}], usage_metadata={'input_tokens': 217, 'output_tokens': 18, 'total_tokens': 235}), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', 

In [49]:
to_replay = states[-3]

to_replay

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather like in LA now?', id='87141516-c120-4c37-9d1c-aba077de68cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 217, 'total_tokens': 235}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7433f4af-3e19-44a6-928c-337cc9fff2f2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_8726256235629355699'}], usage_metadata={'input_tokens': 217, 'output_tokens': 18, 'total_tokens': 235})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef24b26-10ca-6580-8001-c4a3a269d7da'}}, metadata={'source': 'loop'

In [50]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_8726256235629355699'}
Back to the Model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1717753360, \'localtime\': \'2024-06-07 2:42\'}, \'current\': {\'last_updated_epoch\': 1717752600, \'last_updated\': \'2024-06-07 02:30\', \'temp_c\': 15.6, \'temp_f\': 60.1, \'is_day\': 0, \'condition\': {\'text\': \'Mist\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/143.png\', \'code\': 1030}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, \'wind_degree\': 230, \'wind_dir\': \'SW\', \'pressure_mb\': 1012.0, \'pressure_in\': 29.89, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 96, \'cloud\': 100, \'feelslike_c\': 15.6, \'feelsl

### Go back in time and edit

In [51]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]

to_replay.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in LA, accuweather"},
        "id": _id,
    }
]

In [52]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='What is the weather like in LA now?', id='87141516-c120-4c37-9d1c-aba077de68cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 217, 'total_tokens': 235}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7433f4af-3e19-44a6-928c-337cc9fff2f2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_8726256235629355699'}], usage_metadata={'input_tokens': 217, 'output_tokens': 18, 'total_tokens': 235})]}


In [53]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_8726256235629355699'}
Back to the Model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1717753528, \'localtime\': \'2024-06-07 2:45\'}, \'current\': {\'last_updated_epoch\': 1717753500, \'last_updated\': \'2024-06-07 02:45\', \'temp_c\': 15.6, \'temp_f\': 60.1, \'is_day\': 0, \'condition\': {\'text\': \'Mist\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/143.png\', \'code\': 1030}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, \'wind_degree\': 250, \'wind_dir\': \'WSW\', \'pressure_mb\': 1012.0, \'pressure_in\': 29.89, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 96, \'cloud\': 100, \'feelslike_c\': 15.6, \'f

### Add message to a state at a given time

In [54]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]

In [55]:
state_update = {
    "messages": [
        ToolMessage(
            tool_call_id=_id,
            name="tavily_search_results_json",
            content="54 degree celcius",
        )
    ]
}

In [56]:
branch_and_add = abot.graph.update_state(
    to_replay.config, state_update, as_node="action"
)

In [57]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='What is the weather like in LA now?', id='87141516-c120-4c37-9d1c-aba077de68cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8726256235629355699', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 217, 'total_tokens': 235}, 'model_name': 'glm-4-air', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7433f4af-3e19-44a6-928c-337cc9fff2f2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_8726256235629355699'}], usage_metadata={'input_tokens': 217, 'output_tokens': 18, 'total_tokens': 235}), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='c62422a9-7253-4c2f-b8b1-0378ef5a8ee9', tool_call_id='call_8726256235629355699'), AIMe

### Extra Practice
#### Build a small graph
This is a small graph you can tinker with if you want more insight into controlling state memory.

In [58]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [60]:
def node1(state: AgentState) -> dict:
    print(f"node1, count: {state['count']}")
    return {"lnode": "node_1", "count": 1}


def node2(state: AgentState) -> dict:
    print(f"node2, count: {state['count']}")
    return {"lnode": "node_2", "count": 1}

In [61]:
def should_continue(state):
    return state["count"] < 3

In [62]:
builder = StateGraph(AgentState)
builder.add_node(node="Node1", action=node1)
builder.add_node(node="Node2", action=node2)

builder.add_edge(start_key="Node1", end_key="Node2")
builder.add_conditional_edges(
    source="Node2", path=should_continue, path_map={True: "Node1", False: END}
)

builder.set_entry_point(key="Node1")

In [63]:
memory = SqliteSaver.from_conn_string(":memory:")

graph = builder.compile(checkpointer=memory)

#### Run it!

In [64]:
thread = {"configurable": {"thread_id": "1"}}

graph.invoke({"count": 0, "scratch": "hi"}, thread)

node1, count: 0
node2, count: 1
node1, count: 2
node2, count: 3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

#### Look at current state

In [65]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ff-67df-8004-54743f9a692c'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-07T09:58:01.425081+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0fb-6819-8003-8df676c7bbe2'}})

### Look at state history

In [66]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ff-67df-8004-54743f9a692c'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-07T09:58:01.425081+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0fb-6819-8003-8df676c7bbe2'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0fb-6819-8003-8df676c7bbe2'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-07T09:58:01.423450+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0f8-6813-8002-7a81808d3d56'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

In [67]:
states = []

for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ff-67df-8004-54743f9a692c'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0fb-6819-8003-8df676c7bbe2'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0f8-6813-8002-7a81808d3d56'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0f2-6ce4-8001-b02a7e24a5c2'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ed-6e7a-8000-7cfa1e169f8b'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ea-6566-bfff-d82040d04024'}} 0


In [68]:
states[-3]

{'configurable': {'thread_id': '1',
  'thread_ts': '1ef24b46-d0f2-6ce4-8001-b02a7e24a5c2'}}

In [69]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0f2-6ce4-8001-b02a7e24a5c2'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-07T09:58:01.419887+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ed-6e7a-8000-7cfa1e169f8b'}})

#### Go back in Time

In [70]:
graph.invoke(None, states[-3])

node2, count: 1
node1, count: 2
node2, count: 3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [71]:
thread = {"configurable": {"thread_id": "1"}}

for state in graph.get_state_history(thread):
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b52-d829-6e74-8004-a33e663ef2e1'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b52-d825-6fb5-8003-d8a8eac03c81'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b52-d823-6178-8002-a00a2e65870d'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ff-67df-8004-54743f9a692c'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0fb-6819-8003-8df676c7bbe2'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0f8-6813-8002-7a81808d3d56'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0f2-6ce4-8001-b02a7e24a5c2'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ed-6e7a-8000-7cfa1e169f8b'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24b46-d0ea-6566-bfff-d82040d04024'}} 0


In [72]:
thread = {"configurable": {"thread_id": "1"}}

for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b52-d829-6e74-8004-a33e663ef2e1'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-07T10:03:24.299008+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b52-d825-6fb5-8003-d8a8eac03c81'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b52-d825-6fb5-8003-d8a8eac03c81'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-07T10:03:24.297397+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24b52-d823-6178-8002-a00a2e65870d'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

#### Modify State

In [73]:
thread2 = {"configurable": {"thread_id": "2"}}

graph.invoke({"count": 0, "scratch": "hi"}, thread2)

node1, count: 0
node2, count: 1
node1, count: 2
node2, count: 3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [75]:
states2 = []

for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1019-6365-8004-eb0f342f7a86'}} 4
{'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1016-668b-8003-05a896297ef1'}} 3
{'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1014-670c-8002-4a41ca588a18'}} 2
{'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1011-670b-8001-c8eeb42462b5'}} 1
{'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-100e-687b-8000-722c5c156ff5'}} 0
{'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-100c-697d-bfff-da72788b8915'}} 0


In [76]:
save_state = graph.get_state(states2[-3])
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1011-670b-8001-c8eeb42462b5'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-07T10:11:06.501287+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-100e-687b-8000-722c5c156ff5'}})

In [77]:
save_state.values["count"] = -3
save_state.values["scratch"] = "Hello"
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'Hello', 'count': -3}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1011-670b-8001-c8eeb42462b5'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-07T10:11:06.501287+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-100e-687b-8000-722c5c156ff5'}})

In [78]:
graph.update_state(thread2, save_state.values)

{'configurable': {'thread_id': '2',
  'thread_ts': '1ef24b69-3804-6a4b-8005-1e1a73a8e4a3'}}

In [79]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:
        break

    print(state, "\n")

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'Hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b69-3804-6a4b-8005-1e1a73a8e4a3'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': 'Hello', 'count': -3}}}, created_at='2024-06-07T10:13:24.908079+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1019-6365-8004-eb0f342f7a86'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1019-6365-8004-eb0f342f7a86'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-07T10:11:06.504473+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1016-668b-8003-05a896297ef1'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'

#### Try again with `as_node`

In [80]:
graph.update_state(thread2, save_state.values, as_node="Node1")

{'configurable': {'thread_id': '2',
  'thread_ts': '1ef24b6f-00dc-64e2-8006-238bcdee3b49'}}

In [81]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:
        break

    print(state, "\n")

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'Hello', 'count': -2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b6f-00dc-64e2-8006-238bcdee3b49'}}, metadata={'source': 'update', 'step': 6, 'writes': {'Node1': {'lnode': 'node_1', 'scratch': 'Hello', 'count': -3}}}, created_at='2024-06-07T10:16:00.185654+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b69-3804-6a4b-8005-1e1a73a8e4a3'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'Hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b69-3804-6a4b-8005-1e1a73a8e4a3'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': 'Hello', 'count': -3}}}, created_at='2024-06-07T10:13:24.908079+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b64-1019-6365-8004-eb0f342f7a86'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(),

In [82]:
graph.invoke(None, thread2)

node2, count: -2
node1, count: -1
node2, count: 0
node1, count: 1
node2, count: 2


{'lnode': 'node_2', 'scratch': 'Hello', 'count': 3}

In [83]:
for state in graph.get_state_history(thread2):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'Hello', 'count': 3}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b71-fae8-6ecf-800b-920b79d9ff13'}}, metadata={'source': 'loop', 'step': 11, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-07T10:17:20.092323+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b71-fae5-65a9-800a-dd417b7e553f'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'Hello', 'count': 2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b71-fae5-65a9-800a-dd417b7e553f'}}, metadata={'source': 'loop', 'step': 10, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-07T10:17:20.090855+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef24b71-fae1-6aa4-8009-200c1b61c21d'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'Hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2'